In [50]:
import pandas as pd
import numpy as np
import sqlite3

### Data source
CDC: https://data.cdc.gov/Flu-Vaccinations/Vaccines-gov-Flu-vaccinating-provider-locations/bugr-bbfr

### Information
The Vaccines.gov dataset includes provider information for flu vaccine provider locations in the U.S. Vaccines.gov is powered by VaccineFinder.

### Suggestions on building functions
1. Browsing: 1) input `number` **(not that meaningful?)**
                
                ----> number of rows in the dataset 
2. Searching: input `state+city` 

                    ---> find flu vaccine provider in the same city(name, location street, open hours, other information)
               
3. Searching (auto-completion): input `part of provider's name` (loc_name) 
                                
                                ----> whole provider's name + relevant information(location, open hours, other information)

## EDA

In [51]:
vaccines_data = pd.read_csv("F:/Duke MIDS/821/assignment/final_proj/Biostats821_Final_Project/data/Vaccines.gov__Flu_vaccinating_provider_locations.csv", low_memory=False)
vaccines_data.head()


,provider_location_guid,loc_store_no,loc_phone,loc_name,loc_admin_street1,loc_admin_street2,loc_admin_city,loc_admin_state,loc_admin_zip,sunday_hours,...,insurance_accepted,walkins_accepted,provider_notes,searchable_name,in_stock,supply_level,quantity_last_updated,latitude,longitude,category
0,70d24f31-f2c4-4e08-b1b6-afbbd83947a2,Not applicable,(580) 924-4285,BRYAN CHD - DURANT,1524 W. Chuckwa DR,NaN,Durant,OK,74701,NaN,...,True,True,NaN,Flu Shot,True,-1,2022-10-25,34.008567,-96.382197,seasonal
1,9cd97dd4-2196-4585-a1e5-c1ed72e94510,Not applicable,(304) 682-0444,Renegade Pharmacy Inc,18 Logan Street,NaN,Oceana,WV,24870,Closed,...,True,True,Covid 19 vaccinations given Mon.-Wed.-Fri betw...,"Flu Shot (65+, high-dose or adjuvanted)",False,3,2022-12-23,37.692379,-81.634361,seasonal
2,328f435a-3fe8-4262-bd00-17eab759ced2,Not applicable,(931) 647-6561,ST. BETHLEHEM DRUGS,800 WEATHERLY DR SUITE 101A,NaN,CLARKSVILLE,TN,37043,Closed,...,True,True,Please call to schedule an appointment ONLY if...,Flu Shot (Egg free),False,-1,2022-12-08,36.574125,-87.285068,seasonal
3,a499a795-2e5c-4f7f-a3a2-e3042bd0e813,Not applicable,215-735-5600,Center City Pediatrics,1740 South Street,1740 South St Suite 200,Philadelphia,PA,19146,NaN,...,NaN,NaN,NaN,Flu Shot,True,4,2022-12-19,39.944177,-75.171724,seasonal
4,eaf46825-8bba-4f27-ba26-7f9fb787030d,MS1005662,(718) 433-9800,Vida Sana Pharmacy #MS1005662,8820 37th ave,NaN,Jackson Heights,NY,11372-7737,Closed,...,True,True,NaN,"Flu Shot (65+, high-dose or adjuvanted)",True,4,2023-01-02,40.750210,-73.878084,seasonal


In [52]:
vaccines_data.shape

(245118, 28)

In [53]:
# count the number of searchable_name
vaccines_data['searchable_name'].value_counts().to_frame()

,searchable_name
Flu Shot,116706
Flu Shot (Egg free),63137
"Flu Shot (65+, high-dose or adjuvanted)",58364
Flu Nasal Spray,6911


In [54]:
# vaccinate provider in Durham, NC
VAC = vaccines_data.loc[vaccines_data['loc_admin_city'] == 'Durham',:]
VAC.head()

,provider_location_guid,loc_store_no,loc_phone,loc_name,loc_admin_street1,loc_admin_street2,loc_admin_city,loc_admin_state,loc_admin_zip,sunday_hours,...,insurance_accepted,walkins_accepted,provider_notes,searchable_name,in_stock,supply_level,quantity_last_updated,latitude,longitude,category
4108,2dd9f103-5118-42e1-986c-21229835853d,10-4831,9194898160,Sams Club #10-4831,4005 Durham Chapel Hill Blvd,NaN,Durham,NC,27707-2516,NaN,...,True,True,NaN,Flu Nasal Spray,False,-1,2023-03-28,35.967432,-78.958222,seasonal
4985,824308c3-7468-4c8b-90c8-c0a57f674454,1817,(919) 598-0803,Publix Super Markets Inc. #1817,1065 Yunus Road,NaN,Durham,NC,27703-7200,11:00AM - 6:00PM,...,True,True,NaN,Flu Shot,False,-1,2023-04-06,35.940518,-78.853660,seasonal
4986,824308c3-7468-4c8b-90c8-c0a57f674454,1817,(919) 598-0803,Publix Super Markets Inc. #1817,1065 Yunus Road,NaN,Durham,NC,27703-7200,11:00AM - 6:00PM,...,True,True,NaN,Flu Shot,False,-1,2023-04-06,35.940518,-78.853660,seasonal
21086,824308c3-7468-4c8b-90c8-c0a57f674454,1817,(919) 598-0803,Publix Super Markets Inc. #1817,1065 Yunus Road,NaN,Durham,NC,27703-7200,11:00AM - 6:00PM,...,True,True,NaN,Flu Shot (Egg free),False,-1,2023-04-06,35.940518,-78.853660,seasonal
50223,d350fa41-8714-4e81-bed7-a4a310dfce3d,420,919-620-1947,Food Lion #420,3808 Guess Road,NaN,Durham,NC,27705,NaN,...,True,True,NaN,Flu Shot (Egg free),False,-1,2023-04-07,36.058269,-78.928471,seasonal


In [55]:
vaccines_data['category'].value_counts().to_frame()

,category
seasonal,245118


In [56]:
vaccines_data['supply_level'].value_counts().to_frame()


,supply_level
-1,207258
4,28463
3,8088
1,1309


In [57]:
vaccines_data.columns

Index(['provider_location_guid', 'loc_store_no', 'loc_phone', 'loc_name',
       'loc_admin_street1', 'loc_admin_street2', 'loc_admin_city',
       'loc_admin_state', 'loc_admin_zip', 'sunday_hours', 'monday_hours',
       'tuesday_hours', 'wednesday_hours', 'thursday_hours', 'friday_hours',
       'saturday_hours', 'web_address', 'pre_screen', 'insurance_accepted',
       'walkins_accepted', 'provider_notes', 'searchable_name', 'in_stock',
       'supply_level', 'quantity_last_updated', 'latitude', 'longitude',
       'category'],
      dtype='object')

In [58]:
# drop the columns that are not needed 'loc_store_no', 'latitude', 'longitude','category'
vac_new = vaccines_data.drop(['loc_store_no', 'latitude', 'longitude','category','supply_level'], axis=1)
vac_new.shape


(245118, 23)

In [59]:
vac_new['sunday_hours'] = 'sunday_hours:  '+ vac_new['sunday_hours'].astype(str) + '  |  '
vac_new['monday_hours'] = 'monday_hours:  '+ vac_new['monday_hours'].astype(str) + '  |  '
vac_new['tuesday_hours'] = 'tuesday_hours:  '+ vac_new['tuesday_hours'].astype(str) + '  |  '
vac_new['wednesday_hours'] = 'wednesday_hours:  '+ vac_new['wednesday_hours'].astype(str) + '  |  '
vac_new['thursday_hours'] = 'thursday_hours:  '+ vac_new['thursday_hours'].astype(str) + '  |  '
vac_new['friday_hours'] = 'friday_hours:  '+ vac_new['friday_hours'].astype(str) + '  |  '
vac_new['saturday_hours'] = 'saturday_hours:  '+ vac_new['saturday_hours'].astype(str)

# combine the hours into one column
vac_new['opening_hours'] = vac_new['sunday_hours'] + vac_new['monday_hours'] + vac_new['tuesday_hours'] 
+ vac_new['wednesday_hours'] + vac_new['thursday_hours'] + vac_new['friday_hours'] + vac_new['saturday_hours']

0         wednesday_hours:  8:00 AM - 5:00 PM  |  thursd...
1         wednesday_hours:  8:30 AM - 6:30 PM  |  thursd...
2         wednesday_hours:  9:00 AM - 6:00 PM  |  thursd...
3         wednesday_hours:  nan  |  thursday_hours:  nan...
4         wednesday_hours:  9:30 AM - 7:00 PM  |  thursd...
                                ...                        
245113    wednesday_hours:  08:00AM-09:00PM  |  thursday...
245114    wednesday_hours:  08:00AM-08:00PM  |  thursday...
245115    wednesday_hours:  9:00 AM - 7:00 PM  |  thursd...
245116    wednesday_hours:  9:00 AM - 7:00 PM  |  thursd...
245117    wednesday_hours:  8:00 am - 8:00 pm  |  thursd...
Length: 245118, dtype: object

In [60]:
vac_new['opening_hours'].head()

0    sunday_hours:  nan  |  monday_hours:  8:00 AM ...
1    sunday_hours:  Closed  |  monday_hours:  8:30 ...
2    sunday_hours:  Closed  |  monday_hours:  9:00 ...
3    sunday_hours:  nan  |  monday_hours:  nan  |  ...
4    sunday_hours:  Closed  |  monday_hours:  9:30 ...
Name: opening_hours, dtype: object

In [61]:
vac_new.drop(['sunday_hours', 'monday_hours', 'tuesday_hours','wednesday_hours','thursday_hours','friday_hours','saturday_hours'], axis=1, inplace=True)
vac_new.columns

Index(['provider_location_guid', 'loc_phone', 'loc_name', 'loc_admin_street1',
       'loc_admin_street2', 'loc_admin_city', 'loc_admin_state',
       'loc_admin_zip', 'web_address', 'pre_screen', 'insurance_accepted',
       'walkins_accepted', 'provider_notes', 'searchable_name', 'in_stock',
       'quantity_last_updated', 'opening_hours'],
      dtype='object')

In [62]:
# check data type
vac_new.dtypes

provider_location_guid    object
loc_phone                 object
loc_name                  object
loc_admin_street1         object
loc_admin_street2         object
loc_admin_city            object
loc_admin_state           object
loc_admin_zip             object
web_address               object
pre_screen                object
insurance_accepted        object
walkins_accepted          object
provider_notes            object
searchable_name           object
in_stock                    bool
quantity_last_updated     object
opening_hours             object
dtype: object

In [63]:
# save the new dataframe to a csv file
# vac_new.to_csv('vaccines_processed.csv', index=False)

# Subset for NC

In [64]:
# subset for NC
vac_NC = vac_new.loc[vac_new['loc_admin_state'] == 'NC',:]
vac_NC.sample(5)

,provider_location_guid,loc_phone,loc_name,loc_admin_street1,loc_admin_street2,loc_admin_city,loc_admin_state,loc_admin_zip,web_address,pre_screen,insurance_accepted,walkins_accepted,provider_notes,searchable_name,in_stock,quantity_last_updated,opening_hours
100351,f23d78e4-8a83-453a-97c4-0db081473b6d,910-681-1134,Walgreens Co. #9931,8290 MARKET ST,-,WILMINGTON,NC,28411-9388,https://www.walgreens.com,https://www.walgreens.com/findcare/vaccination...,True,True,Appointments are not required to receive a Flu...,Flu Shot,True,2023-04-03,sunday_hours: 10:00AM - 06:00PM | monday_ho...
229056,c14a8aad-bbcd-45bf-8af9-60393bb9374d,(252) 728-4115,"CVS Pharmacy, Inc. #07381",1703 LIVE OAK ST.,NaN,BEAUFORT,NC,28516,https://www.cvs.com/store-locator/details-dire...,https://www.cvs.com/immunizations/get-vaccinated,True,True,It is highly recommended to make an appointmen...,"Flu Shot (65+, high-dose or adjuvanted)",False,2023-04-06,sunday_hours: 10:00AM-06:00PM | monday_hour...
57763,2604d072-3ce1-49cb-bcbd-209e4818b14c,919-775-4361,Walgreens Co. #7479,1956 S HORNER BLVD,-,SANFORD,NC,27330-5841,https://www.walgreens.com,https://www.walgreens.com/findcare/vaccination...,True,True,Appointments are not required to receive a Flu...,Flu Shot,True,2023-04-03,sunday_hours: 09:00AM - 09:00PM | monday_ho...
163516,c073c08e-246a-4fe9-8ce8-6537fbfb938e,(919) 845-5276,"CVS Pharmacy, Inc. #05313",13304 LEESVILLE CHURCH ROAD,NaN,RALEIGH,NC,27617,https://www.cvs.com/store-locator/details-dire...,https://www.cvs.com/immunizations/get-vaccinated,True,True,It is highly recommended to make an appointmen...,Flu Shot,True,2022-11-19,sunday_hours: 10:00AM-06:00PM | monday_hour...
49600,2abcbf1c-f7ae-47bb-bf4d-676e137be4ed,(910) 860-4606,"CVS Pharmacy, Inc. #16231",2056 SKIBO RD,NaN,FAYETTEVILLE,NC,28314,https://www.cvs.com/store-locator/details-dire...,https://www.cvs.com/immunizations/get-vaccinated,True,True,It is highly recommended to make an appointmen...,Flu Shot (Egg free),False,2023-04-06,sunday_hours: 11:00AM-05:00PM | monday_hour...


## Duplicates

In [65]:
print(f"Number of duplicates: {vac_NC.duplicated().sum()}")

# drop duplicates
vac_NC = vac_NC.drop_duplicates()

assert vac_NC.duplicated().sum() == 0
print(f"Number of duplicates now: {vac_NC.duplicated().sum()}")

Number of duplicates: 3198
Number of duplicates now: 0


## Missings

In [66]:
for i in vac_NC.columns:
    print(f"Column {i} has {vac_NC[i].isnull().sum()} null values, percentage: {round(vac_NC[i].isnull().sum()/vac_NC.shape[0], 2)}")

Column provider_location_guid has 0 null values, percentage: 0.0
Column loc_phone has 0 null values, percentage: 0.0
Column loc_name has 0 null values, percentage: 0.0
Column loc_admin_street1 has 0 null values, percentage: 0.0
Column loc_admin_street2 has 4102 null values, percentage: 0.79
Column loc_admin_city has 0 null values, percentage: 0.0
Column loc_admin_state has 0 null values, percentage: 0.0
Column loc_admin_zip has 0 null values, percentage: 0.0
Column web_address has 6 null values, percentage: 0.0
Column pre_screen has 1228 null values, percentage: 0.24
Column insurance_accepted has 6 null values, percentage: 0.0
Column walkins_accepted has 6 null values, percentage: 0.0
Column provider_notes has 1874 null values, percentage: 0.36
Column searchable_name has 0 null values, percentage: 0.0
Column in_stock has 0 null values, percentage: 0.0
Column quantity_last_updated has 0 null values, percentage: 0.0
Column opening_hours has 0 null values, percentage: 0.0


loc_admin_street2 has too many missing values -- drop the column

In [67]:
vac_NC = vac_NC.drop(['loc_admin_street2'], axis=1)

In [68]:
# replace missing values as "Not Available"
vac_NC['pre_screen'] = vac_NC['pre_screen'].fillna('Not Available')
vac_NC['provider_notes'] = vac_NC['provider_notes'].fillna('Not Available')
vac_NC['web_address'] = vac_NC['web_address'].fillna('Not Available')
vac_NC['insurance_accepted'] = vac_NC['insurance_accepted'].fillna('Not Available')
vac_NC['walkins_accepted'] = vac_NC['walkins_accepted'].fillna('Not Available')

assert vac_NC.isnull().sum().sum() == 0

## Format phone number

In [69]:
# format phone number as xxx-xxx-xxxx
vac_NC['loc_phone'] = vac_NC['loc_phone'].str.extractall(r'(\d)').unstack().apply(lambda x: ''.join(x), axis=1)
vac_NC['loc_phone'] = vac_NC['loc_phone'].str.replace(r'(\d{3})(\d{3})(\d{4})', r'\1-\2-\3')
vac_NC.sample(5)

C:\Users\LittlePigPig\AppData\Local\Temp\ipykernel_27524\819198046.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  vac_NC['loc_phone'] = vac_NC['loc_phone'].str.replace(r'(\d{3})(\d{3})(\d{4})', r'\1-\2-\3')


,provider_location_guid,loc_phone,loc_name,loc_admin_street1,loc_admin_city,loc_admin_state,loc_admin_zip,web_address,pre_screen,insurance_accepted,walkins_accepted,provider_notes,searchable_name,in_stock,quantity_last_updated,opening_hours
181726,c14a8aad-bbcd-45bf-8af9-60393bb9374d,252-728-4115,"CVS Pharmacy, Inc. #07381",1703 LIVE OAK ST.,BEAUFORT,NC,28516,https://www.cvs.com/store-locator/details-dire...,https://www.cvs.com/immunizations/get-vaccinated,True,True,It is highly recommended to make an appointmen...,Flu Shot,False,2023-04-06,sunday_hours: 10:00AM-06:00PM | monday_hour...
164209,90764a16-73a7-4ac0-a8c4-5bb2e612c190,704-867-2440,Walmart Inc #10-1385,3000 E Franklin Blvd,Gastonia,NC,28056-9451,https://www.walmart.com/store/1385,Not Available,True,True,Not Available,Flu Shot (Egg free),False,2023-03-28,"sunday_hours: 11:00 AM - 1:30PM, 2:00 PM - 5:..."
220400,201d0417-4484-469c-af57-b67d314e46cd,919-403-8059,Walgreens Co. #11894,1109 W NC HIGHWAY 54,DURHAM,NC,27707-5548,https://www.walgreens.com,https://www.walgreens.com/findcare/vaccination...,True,True,Appointments are not required to receive a Flu...,"Flu Shot (65+, high-dose or adjuvanted)",True,2023-04-03,sunday_hours: 10:00AM - 06:00PM | monday_ho...
3890,07bbf65e-298b-4f67-99da-cab2fbac28a7,704-888-3784,Medical Pharmacy of Locust,236 MARKET ST STE 100,LOCUST,NC,28097,https://healthmartcovidvaccine.com,https://healthmartcovidvaccine.com,True,False,Scheduling an appointment is recommended. Plea...,Flu Shot,True,2023-04-07,sunday_hours: nan | monday_hours: 10:00 AM...
92114,fc45e5b2-1870-45ce-83d3-7cb6139cd451,910-296-0675,"CVS Pharmacy, Inc. #07043",634 S. MAIN ST.,KENANSVILLE,NC,28349,https://www.cvs.com/store-locator/details-dire...,https://www.cvs.com/immunizations/get-vaccinated,True,True,It is highly recommended to make an appointmen...,Flu Shot,True,2022-11-19,sunday_hours: 10:00AM-06:00PM | monday_hour...


In [76]:
# save to a csv file
vac_NC.to_csv('F:/Duke MIDS/821/assignment/final_proj/Biostats821_Final_Project/data/vaccines_NC.csv', index=False)

## Create Database Using SQLite

In [81]:
connection = sqlite3.connect("F:/Duke MIDS/821/assignment/final_proj/Biostats821_Final_Project/database/Flu_Vaccines_Provider_NC.db")
cursor = connection.cursor()
cursor.execute("DROP TABLE IF EXISTS Vaccines")

# create a table

cursor.execute("""CREATE TABLE Vaccines (
guid VARCHAR, phone_number VARCHAR, provider_name VARCHAR, street_address1 VARCHAR,
city VARCHAR, state VARCHAR, zip VARCHAR, website VARCHAR, 
pre_sreening_required VARCHAR, insurance_accepted VARCHAR, walkins_accepted VARCHAR, 
provider_notes VARCHAR, searchable_name VARCHAR, in_stock VARCHAR, 
quantity_last_updated VARCHAR, hours VARCHAR)""") 

# insert data into the table
with open('F:/Duke MIDS/821/assignment/final_proj/Biostats821_Final_Project/data/vaccines_NC.csv', 'r') as f:
    next(f) # Skip the header row.
    for line in f:
        vaccines_line = line.strip().split(',')
        cursor.execute("INSERT INTO Vaccines VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)",
                          vaccines_line[0:16])

connection.commit()
connection.close()

                       

In [83]:
# select data from the table
connection = sqlite3.connect("F:/Duke MIDS/821/assignment/final_proj/Biostats821_Final_Project/database/Flu_Vaccines_Provider_NC.db")
cursor = connection.cursor()
cursor.execute("SELECT * FROM Vaccines LIMIT 5")
data = cursor.fetchall()
print(data[0])

('0a2bcf19-9d62-41e0-b22e-cc6b6c371b63', '910-278-6050', 'Thomas Drugs #3438365', '7917 E OAK ISLAND DR', 'Oak Island', 'NC', '28465', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available', 'Flu Shot (Egg free)', 'False', '2023-01-09', 'sunday_hours:  nan  |  monday_hours:  nan  |  tuesday_hours:  nan  |')


In [84]:
vac_NC.iloc[0,:]

provider_location_guid                 0a2bcf19-9d62-41e0-b22e-cc6b6c371b63
loc_phone                                                      910-278-6050
loc_name                                              Thomas Drugs #3438365
loc_admin_street1                                      7917 E OAK ISLAND DR
loc_admin_city                                                   Oak Island
loc_admin_state                                                          NC
loc_admin_zip                                                         28465
web_address                                                   Not Available
pre_screen                                                    Not Available
insurance_accepted                                            Not Available
walkins_accepted                                              Not Available
provider_notes                                                Not Available
searchable_name                                         Flu Shot (Egg free)
in_stock    